In [1]:
import os
import sys

In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))

In [3]:
sys.path.insert(0, os.path.join(BASE_DIR))

In [4]:
from offline import SparkSessionBase

In [5]:
PYSPARK_PYTHON = "/miniconda2/envs/reco_sys/bin/python"
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON

In [6]:
class OriginArticleData(SparkSessionBase):
    SPARK_APP_NAME = 'originArticle'
    ENABLE_HIVE_SUPPORT = True
    def __init__(self):
        self.spark = self._create_spark_session()

In [7]:
oa = OriginArticleData()

In [8]:
oa.spark.sql("use toutiao")

DataFrame[]

In [9]:
basic_content = oa.spark.sql(
            "select a.article_id, a.channel_id, a.title, b.content from news_article_basic a inner join news_article_content b on a.article_id=b.article_id where a.article_id=116636")


In [11]:
basic_content.show()


+----------+----------+---------------+--------------------+
|article_id|channel_id|          title|             content|
+----------+----------+---------------+--------------------+
|    116636|        18|动态再平衡投资策略历史数据回测|<p>赚钱是个俗气的话题，但又是人...|
+----------+----------+---------------+--------------------+



In [12]:
import pyspark.sql.functions as F
import gc


In [13]:
basic_content.registerTempTable('temparticle')

In [14]:
channel_basic_content = oa.spark.sql('select t.*, n.channel_name from temparticle t left join news_channel n on t.channel_id=n.channel_id')

In [15]:
channel_basic_content.show()

+----------+----------+---------------+--------------------+------------+
|article_id|channel_id|          title|             content|channel_name|
+----------+----------+---------------+--------------------+------------+
|    116636|        18|动态再平衡投资策略历史数据回测|<p>赚钱是个俗气的话题，但又是人...|      python|
+----------+----------+---------------+--------------------+------------+



In [16]:
oa.spark.sql("use article")

DataFrame[]

In [17]:
sentence_df = channel_basic_content.select("article_id", "channel_id", "channel_name", "title", "content", \
                                           F.concat_ws(
                                             ",",
                                             channel_basic_content.channel_name,
                                             channel_basic_content.title,
                                             channel_basic_content.content
                                           ).alias("sentence")
                                          )

In [18]:
sentence_df.show()

+----------+----------+------------+---------------+--------------------+--------------------+
|article_id|channel_id|channel_name|          title|             content|            sentence|
+----------+----------+------------+---------------+--------------------+--------------------+
|    116636|        18|      python|动态再平衡投资策略历史数据回测|<p>赚钱是个俗气的话题，但又是人...|python,动态再平衡投资策略历...|
+----------+----------+------------+---------------+--------------------+--------------------+

